In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read the Excel file
data= pd.read_csv('../input/ecommerce-data/data.csv')
data.head()

In [ ]:
data.info()

In [ ]:
#generate invoice month for each line purchase equal to the first day of the month when the purchase was made
data['InvoiceMonth'] = pd.to_datetime(data['InvoiceDate']).to_numpy().astype('datetime64[M]')
#first invoice month for every customer
data['CohortMonth'] = data.groupby('CustomerID')['InvoiceMonth'].transform('min')
#drop null values
data.dropna(inplace=True)
data.head()

In [ ]:
#drop NaN values
data.dropna()
#compute year and month from Invoice Date
invoice_year= data['InvoiceMonth'].dt.year.astype('int')
invoice_mon= data['InvoiceMonth'].dt.month.astype('int')
#compute year and month from Cohort Date
cohort_year= data['CohortMonth'].dt.year.astype('int')
cohort_mon= data['CohortMonth'].dt.month.astype('int')
#find the differences
diff_year = invoice_year - cohort_year
diff_mon = invoice_mon - cohort_mon
#calculate the cohort index for each invoice
data['CohortIndex'] = diff_year * 12 + diff_mon + 1
data.head()

In [ ]:
#group by cohort month and index and find number of unique customers for each grouping
grouped = data.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].apply(pd.Series.nunique)\
                                                                    .reset_index()
#pivot the data with cohort month as rows and Cohort Index as columns
grouped = grouped.pivot(index='CohortMonth', columns='CohortIndex',  values='CustomerID')
grouped

In [ ]:
#divide each column by value of the first cohort size to find retention rate
size = grouped.iloc[:,0]
retention_table = grouped.divide(size, axis=0)
#compute the percentage
retention_table.round(3) * 100